In [64]:
# import required python modules
import pandas as pd
import os.path as os_path
import csv
import os
import re

In [65]:
# set global variables for start and end year -- these must be data type str
START_YEAR = '2015'
END_YEAR = '2022'

In [66]:
import numpy
print(numpy.__version__)
print(pd.__version__)
import sys
print(sys.version)

1.24.3
1.5.1
3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [67]:
# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory:", cwd)


Current working directory: C:\Users\user\Downloads


In [68]:
DATA_DIR = '..\data'
OUTPUT_DIR = '..\output'
PATEX_DIR = 'C:\\Users\\user\\Downloads\\BulkData2023'
PATENTSVIEW_DIR = "C:\\Users\\user\\Downloads\\data_patentsview_20231231"
OUTPUT_FNAME = 'nj_univ_appl_inv_gender.csv'

In [69]:
def id_rutgers_stevens_njit(df, text_col, output_col_suffix=''):
    '''Create columns to identify if text in 'text_col' is associated with Rutgers, Stevens, or NJIT

    This function is tailored for Rutgers University, Stevens Institute of Technology, and
    New Jersey Institute of Technology. Use a set of keywords (whole words or simplified) to
    identify if the text in 'text_col' contains the keywords assocated with each target
    university. A binary output column, one for each target university, identifies the result
    as True or False; to increase flexibility, add a suffix to the output columns.

    Rutgers = rutgers AND (univer OR jersey)
    Stevens = stevens AND institute
    NJIT = jersey AND institute AND technol

    :param df (dataframe): dataframe to analyze
    :param text_col (str): column containing text w/names
    :param output_col_suffix (str): suffix to add to end of output columns (default='')
    :return df (dataframe): dataframe with binary output columns indicating university
    
    # lowercase text in text_col
    df[text_col] = df[text_col].str.lower()
    # analyze Rutgers
    df['rutgers_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('rutgers' in x) and (('univer' in x) or ('jersey' in x)))
    # analyze Stevens
    df['stevens_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('stevens' in x) and ('institute' in x) and ('tech' in x))
    # analyze NJIT
    df['njit_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('jersey' in x) and ('institute' in x) and ('technol' in x))

    #analyse princeton
    df['princeton_'+output_col_suffix] = df[text_col].apply(lambda x: ('princeton' in x) and ('univer' in x))

    #analyse rowan
    df['rowan_'+output_col_suffix] = df[text_col].apply(lambda x: ('rowan' in x) and ('univer' in x))

    #analyse NJCU
    df['NJCU_'+output_col_suffix] = df[text_col].apply(lambda x: ('jersey' in x) and ('city' in x) and ('univer' in x))

    #analyse seton
    df['seton_'+output_col_suffix] = df[text_col].apply(lambda x: ('seton' in x) and ('hall' in x))

    #analyse saint peter's
    df['saintPeters_'+output_col_suffix] = df[text_col].apply(lambda x: ('saint') and ('peter' in x) and ('univer' in x))
    
    # return updated dataframe
    return df
    '''
   
    patterns = {
        'rutgers': r'\b(rutgers(?:,\s*the\s+state\s+university(?:\s+of\s+new\s+jersey)?)?)\b',
        'stevens': r'\b(stevens\s+insti(tute)?)\b',
        'njit': r'\b(jersey\s+insti(tute)?\s+of\s+tech(nology)?)\b',
        'princeton': r'\b(princeton\s+univer(sity)?)\b',
        'rowan': r'\b(rowan\s+univer(sity)?)\b',
        'njcu': r'\b(new\s+jersey\s+city\s+univer(sity)?)\b',
        'setonHall': r'\b(seton\s+hall\s+univer(sity)?)\b',
        'saintPeters': r'\b(sa(in)t\s*peter\s+univer(sity)?)\b',
        'columbia': r'\b(columbia\s+university)\b'
    }

    # Apply each regex pattern to the text column and create a new binary column
    for university, pattern in patterns.items():
        column_name = university + '_' + output_col_suffix
        #df[column_name] = df[text_col].apply(lambda x: bool(re.search(pattern, str(x), re.IGNORECASE)))
        df[column_name] = df[text_col].apply(lambda x: bool(re.findall(pattern, str(x), re.IGNORECASE)))
        print(f'Created column: {column_name}')
        
    return df


In [70]:
def add_appl_owner_to_dict(appl_dict, appl_id, owner):
    '''Add the owner of an application to a dictionary.
    
    The dictionary key is an application number, and values are a set of owners for that application. 
    The use of sets precludes duplicates. Use data type to ignore blank values (e.g., NaN for an owner). 
    Note: we do not use a defaultdict to simplify the later conversion to a pandas dataframe.
    :param appl_dict (dict of sets): initial application owner dictionary 
    :param appl_id (str): application number
    :param owner (str): either an individual applicant or assignee of an application
    :return appl_dict (dict of sets): updated applicaton owner dictionary    
    '''
    # check if input application id is a string
    if type(appl_id)==str:
        # if application id is not in dictionary keys then add it to the dict w/empty set
        if appl_id not in appl_dict.keys(): appl_dict[appl_id] = set()
        # if input owner is a string, add to set for that application 
        if type(owner)==str: appl_dict[appl_id].add(owner)
    return appl_dict

In [71]:
# set filepath to PatEx applicant data csv file
filepath = os_path.join(PATEX_DIR, 'all_applicants.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\all_applicants.csv


In [72]:
with open(filepath, 'r') as file:
    for _ in range(2):
        print(file.readline(), end='')

application_number,applicant_organization,applicant_name_first,applicant_name_middle,applicant_name_last,applicant_city_name,applicant_region_code,applicant_country_code
10470614,,Societe Franco-Belge de Fabrication de,,Combustile - FBFC,Courbevoie,,FR


In [73]:
applicant_df = pd.read_csv(filepath,
                           header=0,
                           dtype=str,
                           keep_default_na=False)

In [74]:
applicant_df.loc[applicant_df['application_number'] == 17976614]

,application_number,applicant_organization,applicant_name_first,applicant_name_middle,applicant_name_last,applicant_city_name,applicant_region_code,applicant_country_code


In [75]:
# filter data to keep only US and NJ applicants
print('Number of rows before US NJ filter: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[applicant_df['applicant_region_code'].isin(['NJ', 'NY'])]

print('Number of rows after US NJ filter: {:,}'.format(len(applicant_df)))

Number of rows before US NJ filter: 6,752,288
Number of rows after US NJ filter: 447,338


In [76]:
# create column for application series code
applicant_df['series'] = applicant_df['application_number'].apply(lambda x: x[0:2])

In [77]:
# filter data to remove PCT applications -- begins with "PCT" and hence the series from above = "PC"
print('Number of rows before PCT filter: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[applicant_df['series']!='PC']

print('Number of rows after PCT filter: {:,}'.format(len(applicant_df)))

Number of rows before PCT filter: 447,338
Number of rows after PCT filter: 255,482


In [78]:
# create combined applicant string - each column separated by a space
# (note: using the python .join() function would be more complicated due to our using dataframes;
#        hence, use a simple concatenation)
applicant_df['applicant_combined'] = applicant_df['applicant_organization'] + ' ' + \
                                     applicant_df['applicant_name_first'] + ' ' + \
                                     applicant_df['applicant_name_middle'] + ' ' + \
                                     applicant_df['applicant_name_last']

In [79]:
applicant_df['count'] = applicant_df['applicant_organization'].str.split(',').apply(lambda x: len([item for item in x if 'INC' not in item]))

In [80]:
#applicant_df[['applicant_combined']].to_excel('output_column.xlsx', index=False)

In [81]:
# identify applications having Rutgers, Stevens, or NJIT
applicant_df = id_rutgers_stevens_njit(applicant_df, 'applicant_combined', output_col_suffix='app')

Created column: rutgers_app
Created column: stevens_app
Created column: njit_app
Created column: princeton_app
Created column: rowan_app
Created column: njcu_app
Created column: setonHall_app
Created column: saintPeters_app
Created column: columbia_app


In [83]:
# summarize results
print('Number of Rutgers applications: {:,}'.format(len(applicant_df.loc[applicant_df['rutgers_app']==True])))
print('Number of Stevens applications: {:,}'.format(len(applicant_df.loc[applicant_df['stevens_app']==True])))
print('Number of NJIT applications: {:,}'.format(len(applicant_df.loc[applicant_df['njit_app']==True])))
print('Number of princeton applications: {:,}'.format(len(applicant_df.loc[applicant_df['princeton_app']==True])))
print('Number of rowan applications: {:,}'.format(len(applicant_df.loc[applicant_df['rowan_app']==True])))
print('Number of NJCU applications: {:,}'.format(len(applicant_df.loc[applicant_df['njcu_app']==True])))
print('Number of seton applications: {:,}'.format(len(applicant_df.loc[applicant_df['setonHall_app']==True])))
print('Number of saintPeters applications: {:,}'.format(len(applicant_df.loc[applicant_df['saintPeters_app']==True])))
print('Number of Columbia applications: {:,}'.format(len(applicant_df.loc[applicant_df['columbia_app']==True])))

Number of Rutgers applications: 1,137
Number of Stevens applications: 208
Number of NJIT applications: 230
Number of princeton applications: 745
Number of rowan applications: 124
Number of NJCU applications: 0
Number of seton applications: 14
Number of saintPeters applications: 0
Number of Columbia applications: 2,172


In [85]:
#print(applicant_df[applicant_df['columbia_app']==True])

In [86]:
# summarize the applicant names associated with the target universities
for univ in ['rutgers_app', 'stevens_app', 'njit_app','princeton_app','rowan_app','njcu_app','setonHall_app','saintPeters_app','columbia_app']:
    print('Applicant names for {}'.format(univ))
    print('----------------------')
    df = applicant_df.loc[applicant_df[univ]==True]
    print(df['applicant_combined'].value_counts())
    print()

Applicant names for rutgers_app
----------------------
Rutgers, The State University of New Jersey              823
RUTGERS, THE STATE UNIVERSITY OF NEW JERSEY              220
Rutgers, the State University of New Jersey               46
Rutgers University                                         7
Rutgers, The State University of New Jersey; Offic         5
Rutgers, The State University                              5
   Rutgers, The State University of New Jersey             4
Rutgers,The State University of New Jersey                 4
RUTGERS THE STATE UNIVERSITY OF NEW JERSEY                 3
Rutgers, the State University                              2
 RUTGERS, THE STATE UNIVERSITY OF  NEW JERSEY              2
RUTGERS UNIVERSITY                                         2
RUTGERS, THE STATE UNIVERSITY                              2
Rutgers, the State Univesity of New Jersey                 1
RUTGERS, the State University of New Jersey                1
Rutgers, The States University

In [87]:
## keep only observations w/target universities
print('Number of rows before removing non-target universities: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[(applicant_df['rutgers_app']==True) | \
                                (applicant_df['stevens_app']==True) | \
                                (applicant_df['njit_app']==True) | \
                                (applicant_df['princeton_app']==True) | \
                                (applicant_df['rowan_app']==True) | \
                                (applicant_df['njcu_app']==True) | \
                                (applicant_df['setonHall_app']==True) | \
                                (applicant_df['saintPeters_app']==True) | \
                                (applicant_df['columbia_app']==True)]

print('Number of rows after removing non-target universities: {:,}'.format(len(applicant_df)))

Number of rows before removing non-target universities: 255,482
Number of rows after removing non-target universities: 4,630


In [88]:
# keep only the columns we need
applicant_df = applicant_df[['application_number','rutgers_app', 'stevens_app', \
                             'njit_app','princeton_app','rowan_app','njcu_app','setonHall_app','saintPeters_app','columbia_app']]

In [90]:
# create a consoldiated university variable
applicant_df.loc[applicant_df['rutgers_app']==True,'univ'] = 'Rutgers'
applicant_df.loc[applicant_df['stevens_app']==True,'univ'] = 'Stevens'
applicant_df.loc[applicant_df['njit_app']==True,'univ'] = 'NJIT'
applicant_df.loc[applicant_df['princeton_app']==True,'univ'] = 'Princeton'
applicant_df.loc[applicant_df['rowan_app']==True,'univ'] = 'Rowan'
applicant_df.loc[applicant_df['njcu_app']==True,'univ'] = 'NJCU'
applicant_df.loc[applicant_df['setonHall_app']==True,'univ'] = 'Seton'
applicant_df.loc[applicant_df['saintPeters_app']==True,'univ'] = 'saintPeters'
applicant_df.loc[applicant_df['columbia_app']==True,'univ'] = 'Columbia'
# print results
print(applicant_df['univ'].value_counts())

Columbia     2172
Rutgers      1137
Princeton     745
NJIT          230
Stevens       208
Rowan         124
Seton          14
Name: univ, dtype: int64


In [91]:
# define columns we want to use (see above)
'''
col_g_assignee  = ['patent_id','disambig_assignee_organization',
                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']
col_pg_assignee = ['pgpub_id','disambig_assignee_organization',
                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']
'''

"\ncol_g_assignee  = ['patent_id','disambig_assignee_organization',\n                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']\ncol_pg_assignee = ['pgpub_id','disambig_assignee_organization',\n                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']\n"

In [25]:
# set seperator for the data files; for tsv file, use '/t'
seperator = '\t'

In [26]:
# # open assignee data for patents and PGPubs as pandas dataframes
# # - identify the header as row 0 (the first row)
# # - specify columns we want to read via usecols option
# # - use seperator as specified above
# # - infer compression (default but specify this option anyway)
# # - ensure all columns are read in as strings
# # - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
# patent_assignee_df = pd.read_csv(os_path.join(PATENTSVIEW_DIR,'g_assignee_disambiguated.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_g_assignee, 
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)
# pgpub_assignee_df =  pd.read_csv(os_path.join(PATENTSVIEW_DIR,'pg_assignee_disambiguated.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_pg_assignee,
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)

In [27]:
# # as before, create a string combining all assignee name information
# patent_assignee_df['assignee_combined'] = patent_assignee_df['disambig_assignee_organization'] + ' ' + \
#                                           patent_assignee_df['disambig_assignee_individual_name_first'] + ' ' + \
#                                           patent_assignee_df['disambig_assignee_individual_name_last']
# pgpub_assignee_df['assignee_combined']  = pgpub_assignee_df['disambig_assignee_organization'] + ' ' + \
#                                           pgpub_assignee_df['disambig_assignee_individual_name_first'] + ' ' + \
#                                           pgpub_assignee_df['disambig_assignee_individual_name_last']

In [28]:
# # identify if target universities -- patents 
# patent_assignee_df = id_rutgers_stevens_njit(patent_assignee_df, 
#                                              'assignee_combined', 
#                                              output_col_suffix='asg_patent')

In [29]:
# # identify if target universities -- pgpubs
# pgpub_assignee_df  = id_rutgers_stevens_njit(pgpub_assignee_df, 
#                                              'assignee_combined', 
#                                              output_col_suffix='asg_pgpub')

In [30]:
# # summarize the assignee names associated with the target universities - patents
# for univ in ['rutgers_asg_patent', 'stevens_asg_patent', 'njit_asg_patent','princeton_asg_patent','rowan_asg_patent','seton_asg_patent','NJCU_asg_patent','saintPeters_asg_patent']:
#     print('Assignee names for {}'.format(univ))
#     print('----------------------')
#     df = patent_assignee_df.loc[patent_assignee_df[univ]==True]
#     print(df['disambig_assignee_organization'].value_counts())
#     print()

In [31]:
# # summarize the assignee names associated with the target universities - pgpubs
# for univ in ['rutgers_asg_pgpub', 'stevens_asg_pgpub', 'njit_asg_pgpub','princeton_asg_pgpub','rowan_asg_pgpub','NJCU_asg_pgpub','seton_asg_pgpub','saintPeters_asg_pgpub']:
#     print('Assignee names for {}'.format(univ))
#     print('----------------------')
#     df = pgpub_assignee_df.loc[pgpub_assignee_df[univ]==True]
#     print(df['disambig_assignee_organization'].value_counts())
#     print()

In [32]:
# # keep only observations w/target universities - patents
# print('PATENTS')
# print('Number of rows before removing non-target universities: {:,}'.format(len(patent_assignee_df)))

# patent_assignee_df = patent_assignee_df.loc[(patent_assignee_df['rutgers_asg_patent']==True) | \
#                                             (patent_assignee_df['stevens_asg_patent']==True) | \
#                                             (patent_assignee_df['njit_asg_patent']==True)    | \
#                                             (patent_assignee_df['princeton_asg_patent']==True) | \
#                                             (patent_assignee_df['rowan_asg_patent']==True) | \
#                                             (patent_assignee_df['NJCU_asg_patent']==True) | \
#                                             (patent_assignee_df['seton_asg_patent']==True) | \
#                                             (patent_assignee_df['saintPeters_asg_patent']==True)]
# print('Number of rows after removing non-target universities: {:,}'.format(len(patent_assignee_df)))

In [33]:
# # keep only observations w/target universities - pgpubs
# print('PGPUBS')
# print('Number of rows before removing non-target universities: {:,}'.format(len(pgpub_assignee_df)))

# pgpub_assignee_df = pgpub_assignee_df.loc[(pgpub_assignee_df['rutgers_asg_pgpub']==True) | \
#                                           (pgpub_assignee_df['stevens_asg_pgpub']==True) | \
#                                           (pgpub_assignee_df['njit_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['rowan_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['princeton_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['NJCU_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['seton_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['saintPeters_asg_pgpub']==True)]

# print('Number of rows after removing non-target universities: {:,}'.format(len(pgpub_assignee_df)))

In [34]:
# # keep only the columns we need 
# patent_assignee_df = patent_assignee_df[['patent_id','rutgers_asg_patent','stevens_asg_patent','njit_asg_patent','princeton_asg_patent','rowan_asg_patent','NJCU_asg_patent','seton_asg_patent','saintPeters_asg_patent']]
# pgpub_assignee_df = pgpub_assignee_df[['pgpub_id','rutgers_asg_pgpub','stevens_asg_pgpub','njit_asg_pgpub','rowan_asg_pgpub','princeton_asg_pgpub','NJCU_asg_pgpub','seton_asg_pgpub','saintPeters_asg_pgpub']]

In [35]:
# # create a consoldiated university variable - patents
# patent_assignee_df.loc[patent_assignee_df['rutgers_asg_patent']==True,'univ_asg_patent'] = 'Rutgers'
# patent_assignee_df.loc[patent_assignee_df['stevens_asg_patent']==True,'univ_asg_patent'] = 'Stevens'
# patent_assignee_df.loc[patent_assignee_df['njit_asg_patent']==True,'univ_asg_patent'] = 'NJIT'
# patent_assignee_df.loc[patent_assignee_df['princeton_asg_patent']==True,'univ_asg_patent'] = 'Princeton'
# patent_assignee_df.loc[patent_assignee_df['rowan_asg_patent']==True,'univ_asg_patent'] = 'Rowan'
# patent_assignee_df.loc[patent_assignee_df['NJCU_asg_patent']==True,'univ_asg_patent'] = 'NJCU'
# patent_assignee_df.loc[patent_assignee_df['seton_asg_patent']==True,'univ_asg_patent'] = 'Seton'
# patent_assignee_df.loc[patent_assignee_df['saintPeters_asg_patent']==True,'univ_asg_patent'] = 'SaintPeters'

# # print results 
# print('PATENTS')
# print(patent_assignee_df['univ_asg_patent'].value_counts())

In [36]:
# # create a consoldiated university variable - pgpubs
# pgpub_assignee_df.loc[pgpub_assignee_df['rutgers_asg_pgpub']==True,'univ_asg_pgpub'] = 'Rutgers'
# pgpub_assignee_df.loc[pgpub_assignee_df['stevens_asg_pgpub']==True,'univ_asg_pgpub'] = 'Stevens'
# pgpub_assignee_df.loc[pgpub_assignee_df['njit_asg_pgpub']==True,'univ_asg_pgpub'] = 'NJIT'
# pgpub_assignee_df.loc[pgpub_assignee_df['princeton_asg_pgpub']==True,'univ_asg_pgpub'] = 'Princeton'
# pgpub_assignee_df.loc[pgpub_assignee_df['rowan_asg_pgpub']==True,'univ_asg_pgpub'] = 'Rowan'
# pgpub_assignee_df.loc[pgpub_assignee_df['NJCU_asg_pgpub']==True,'univ_asg_pgpub'] = 'NJCU'
# pgpub_assignee_df.loc[pgpub_assignee_df['seton_asg_pgpub']==True,'univ_asg_pgpub'] = 'Seton'
# pgpub_assignee_df.loc[pgpub_assignee_df['saintPeters_asg_pgpub']==True,'univ_asg_pgpub'] = 'SaintPeters'

# # print results 
# print('PGPUBS')
# print(pgpub_assignee_df['univ_asg_pgpub'].value_counts())

In [37]:
# # define columns we want to use (see above)
# col_g_appln = ['patent_id','application_id']
# col_pg_appln = ['pgpub_id','application_id']

In [92]:
# set separator for .tsv file
seperator = '\t'

In [39]:
# # open data for patents and PGPubs as pandas dataframes
# # - identify the header as row 0 (the first row)
# # - specify columns we want to read via usecols option
# # - use same seperator as above
# # - infer compression (default but specify this option anyway)
# # - ensure all columns are read in as strings
# # - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
# patent_appln_df = pd.read_csv(os_path.join(PATENTSVIEW_DIR,'g_application.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_g_appln, 
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)
# pgpub_appln_df =  pd.read_csv(os_path.join(PATENTSVIEW_DIR,'pg_published_application.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_pg_appln,
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)

In [40]:
# # left join assignee dataframe (left table) to application dataframe (right table) -- patents 
# # use indicator=True to filter results 
# patent_assignee_df = patent_assignee_df.merge(patent_appln_df, 
#                                               how='left', 
#                                               on='patent_id', 
#                                               indicator=True)
# # print merge results 
# patent_assignee_df['_merge'].value_counts()

In [41]:
# # keep _merge=='both'
# patent_assignee_df = patent_assignee_df.loc[patent_assignee_df['_merge']=='both']
# # drop _merge column
# patent_assignee_df.drop(columns=['_merge'], inplace=True)

In [42]:
# # left join assignee dataframe (left table) to application dataframe (right table) -- pgpubs 
# # use indicator=True to filter results 
# pgpub_assignee_df = pgpub_assignee_df.merge(pgpub_appln_df, 
#                                             how='inner', 
#                                             on='pgpub_id', 
#                                             indicator=True)
# # print merge results 
# pgpub_assignee_df['_merge'].value_counts()

In [43]:
# # keep _merge=='both'
# pgpub_assignee_df = pgpub_assignee_df.loc[pgpub_assignee_df['_merge']=='both']
# # drop _merge column
# pgpub_assignee_df.drop(columns=['_merge'], inplace=True)

In [44]:
# # print size of assignee dataframes 
# print('Number patents having target universities as assignee: {:,}'.format(len(patent_assignee_df)))
# print('Number PGPubs having target universities as assignee: {:,}'.format(len(pgpub_assignee_df)))

In [94]:
# initialize the dictionary
appl_dict = {}

In [95]:
# process applicant data by looping through each row of the dataframe
# and calling the function add_appl_owner_to_dict 
for i in range(0,len(applicant_df)): 
    appl_dict = add_appl_owner_to_dict(appl_dict, 
                                       applicant_df.iloc[i]['application_number'], 
                                       applicant_df.iloc[i]['univ'])

In [47]:
# # similarly process patent assignee data; note different column names 
# for i in range(0,len(patent_assignee_df)): 
#     appl_dict = add_appl_owner_to_dict(appl_dict, 
#                                        patent_assignee_df.iloc[i]['application_id'], 
#                                        patent_assignee_df.iloc[i]['univ_asg_patent'])

In [48]:
# # similarly process pgpub assignee data; note different column names 
# for i in range(0,len(pgpub_assignee_df)): 
#     appl_dict = add_appl_owner_to_dict(appl_dict, 
#                                        pgpub_assignee_df.iloc[i]['application_id'], 
#                                        pgpub_assignee_df.iloc[i]['univ_asg_pgpub'])

In [96]:
# to faciliate creating a dataframe, create two lists, one for application IDs and another for universities, 
# where the set of university names is converted to a list
appl_list = []
univ_list = []
for key, value in appl_dict.items():
    appl_list.append(key)
    univ_list.append(list(value))
# verify length of the two list are the same 
n_appl_list = len(appl_list)
n_univ_list = len(univ_list)
print('Length of application ID list: {:,}'.format(n_appl_list))
print('Length of university list: {:,}'.format(n_univ_list))
if n_appl_list!=n_univ_list: 
    raise ValueError('# applications do not equal length of university list')

Length of application ID list: 4,591
Length of university list: 4,591


In [97]:
# create dataframe 
nj_univ_df = pd.DataFrame({'application_number': appl_list, 'univ_list': univ_list})

In [98]:
# create column for application series code
nj_univ_df['series'] = nj_univ_df['application_number'].apply(lambda x: x[0:2]) 

In [99]:
# filter data to remove PCT applications -- begins with "PCT" and hence the series from above = "PC"
print('Number of rows before PCT filter: {:,}'.format(len(nj_univ_df)))
nj_univ_df = nj_univ_df.loc[nj_univ_df['series']!='PC']
print('Number of rows after PCT filter: {:,}'.format(len(nj_univ_df)))

Number of rows before PCT filter: 4,591
Number of rows after PCT filter: 4,591


In [100]:
# print series -- value_counts (pandas 1.5) outputs a pandas series whose index are the series codes
nj_univ_df['series'].value_counts().sort_index()

12      1
13    189
14    765
15    696
16    750
17    736
18     46
61    109
62    917
63    382
Name: series, dtype: int64

In [101]:
# print universities -- these will include those with multiple university owners 
nj_univ_df['univ_list'].value_counts().sort_index()

[Columbia]               2163
[Columbia, Rutgers]         2
[NJIT]                    223
[NJIT, Rutgers]             7
[Princeton]               737
[Princeton, Columbia]       1
[Princeton, Rutgers]        4
[Princeton, Stevens]        2
[Rowan]                   122
[Rowan, Rutgers]            2
[Rutgers]                1108
[Seton]                    14
[Stevens]                 187
[Stevens, Columbia]         5
[Stevens, Rutgers]         14
Name: univ_list, dtype: int64

In [102]:
# to illustrate the next step, look at application '14777191' as an example
print('Before pandas explode()')
print(nj_univ_df.loc[nj_univ_df['application_number']=='14777191'].transpose())

Before pandas explode()
                        1069
application_number  14777191
univ_list             [NJIT]
series                    14


In [103]:
# use pandas explode() to break apart multiple universities into seperate rows (keeping same application ID)
print('Number rows prior to breaking apart university lists: {:,}'.format(len(nj_univ_df)))
nj_univ_df = nj_univ_df.explode('univ_list')
print('Number rows after breaking apart university lists: {:,}'.format(len(nj_univ_df)))

Number rows prior to breaking apart university lists: 4,591
Number rows after breaking apart university lists: 4,628


In [104]:
print('After pandas explode()')
print(nj_univ_df.loc[nj_univ_df['application_number']=='14777191'].transpose())

After pandas explode()
                        1069
application_number  14777191
univ_list               NJIT
series                    14


In [105]:
# examine result universities -- all should be singular 
nj_univ_df['univ_list'].value_counts().sort_index()

Columbia     2171
NJIT          230
Princeton     744
Rowan         124
Rutgers      1137
Seton          14
Stevens       208
Name: univ_list, dtype: int64

In [106]:
# rename univ_list column to univ
nj_univ_df.rename(columns={'univ_list': 'univ'}, inplace=True)

In [107]:
# # delete unneeded dataframes and data structures 
# del applicant_df, patent_assignee_df, pgpub_assignee_df
# del appl_dict, appl_list, univ_list

In [108]:
# look at first 5 observations (do not use transpose to preserve column orientation)
# note that university names are not in a list 
nj_univ_df.head(5)

,application_number,univ,series
0,17435954,Rutgers,17
1,17914647,Rutgers,17
2,18005581,Columbia,18
3,18005188,Princeton,18
4,18006212,Columbia,18


In [109]:
# look at the first 6 observations (3 applications) having more than one university owner 
# (option keep=False tags all duplicate observation, not just the first or last)
# (note: this command does not change the underlying dataset - duplicates are not dropped)
nj_univ_df.loc[nj_univ_df.duplicated(subset=['application_number'], keep=False)].head(6)

,application_number,univ,series
462,14348197,Stevens,14
462,14348197,Rutgers,14
948,62170455,Stevens,62
948,62170455,Rutgers,62
978,14751086,Stevens,14
978,14751086,Rutgers,14


In [110]:
# set filepath to PatEx application data csv file
filepath = os_path.join(PATEX_DIR, 'application_data.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\application_data.csv


In [111]:
# identify columns we want to use
col_list = ['application_number','filing_date','application_invention_type',
            'earliest_pgpub_number','earliest_pgpub_date','patent_number','patent_issue_date',
            'invention_title','small_entity_indicator','aia_first_to_file']

# open csv file as a pandas dataframe
# - identify the header as row 0 (the first row)
# - use only selected columns
# - ensure all columns are read in as strings (important for the application number)
# - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
appl_df = pd.read_csv(filepath,
                      header=0,
                      usecols=col_list,
                      dtype=str,
                      keep_default_na=False)

In [112]:
appl_df.head()

,application_number,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file
0,04453098,,,,,,,,UNDISCOUNTED,
1,04544040,,,,,,,,UNDISCOUNTED,
2,04577552,,,,,,,,UNDISCOUNTED,
3,04589670,,,,,,,,UNDISCOUNTED,
4,04578318,,,,,,,,UNDISCOUNTED,


In [113]:
# create column for filing year
appl_df['filing_year'] = appl_df['filing_date'].apply(lambda x: x[0:4])

In [114]:
appl_df.head()

,application_number,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,04453098,,,,,,,,UNDISCOUNTED,,
1,04544040,,,,,,,,UNDISCOUNTED,,
2,04577552,,,,,,,,UNDISCOUNTED,,
3,04589670,,,,,,,,UNDISCOUNTED,,
4,04578318,,,,,,,,UNDISCOUNTED,,


In [115]:
# filter data to keep applications between start and end dates

print('Number of rows before date filter: {:,}'.format(len(appl_df)))

appl_df = appl_df.loc[(appl_df['filing_year']>=START_YEAR) & \
                      (appl_df['filing_year']<=END_YEAR)]

print('Number of rows after date filter: {:,}'.format(len(appl_df)))


Number of rows before date filter: 14,100,378
Number of rows after date filter: 4,246,035


In [116]:
# left join nj_univ_df (left table) to appl_df (right table); use indicator=True to filter results 
nj_univ_df = nj_univ_df.merge(appl_df, 
                                how='left', 
                                on='application_number', 
                                indicator=True)

In [117]:
nj_univ_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year,_merge
0,17435954,Rutgers,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,17914647,Rutgers,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,18005581,Columbia,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,18005188,Princeton,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,18006212,Columbia,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [118]:
merge_results_series = nj_univ_df['_merge'].value_counts()
print(merge_results_series)

print('\nMerge results: ')
print('Number appl w/target NJ university applicants within date range: {:,}'.
      format(merge_results_series['both']))
print('Number appl w/target NJ university applicants outside date range (or no appl data): {:,}'.
      format(merge_results_series['left_only']))

both          3816
left_only      812
right_only       0
Name: _merge, dtype: int64

Merge results: 
Number appl w/target NJ university applicants within date range: 3,816
Number appl w/target NJ university applicants outside date range (or no appl data): 812


In [119]:
# keep _merge=='both'
nj_univ_df = nj_univ_df.loc[nj_univ_df['_merge']=='both']

In [120]:
nj_univ_df['univ'].value_counts()

Columbia     1763
Rutgers       948
Princeton     618
NJIT          193
Stevens       168
Rowan         118
Seton           8
Name: univ, dtype: int64

In [121]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year,_merge
797,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,SMALL,false,2015,both
798,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,SMALL,false,2015,both
799,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015,both
800,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015,both
801,14591327,Columbia,14,2015-01-07,Utility,US20150220832A1,2015-08-06,9171249,2015-10-27,SYSTEMS AND METHODS FOR IDENTIFICATION OF SPIK...,SMALL,false,2015,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4608,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022,both
4609,18090083,Columbia,18,2022-12-28,Utility,US20230148096A1,2023-05-11,,,YEAST-BASED VACCINES,SMALL,Other,2022,both
4610,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022,both
4611,18091615,Rowan,18,2022-12-30,Utility,US20230140691A1,2023-05-04,,,"OPTICALLY CLEAR, IN-SITU FORMING BIODEGRADABLE...",SMALL,true,2022,both


In [122]:
# drop _merge column
nj_univ_df.drop(columns=['_merge'], inplace=True)

In [123]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
797,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,SMALL,false,2015
798,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,SMALL,false,2015
799,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015
800,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015
801,14591327,Columbia,14,2015-01-07,Utility,US20150220832A1,2015-08-06,9171249,2015-10-27,SYSTEMS AND METHODS FOR IDENTIFICATION OF SPIK...,SMALL,false,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4608,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022
4609,18090083,Columbia,18,2022-12-28,Utility,US20230148096A1,2023-05-11,,,YEAST-BASED VACCINES,SMALL,Other,2022
4610,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022
4611,18091615,Rowan,18,2022-12-30,Utility,US20230140691A1,2023-05-04,,,"OPTICALLY CLEAR, IN-SITU FORMING BIODEGRADABLE...",SMALL,true,2022


In [124]:
# reset index; do not insert previous index into dataframe
nj_univ_df.reset_index(drop=True, inplace=True)

In [125]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,SMALL,false,2015
1,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,SMALL,false,2015
2,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015
3,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015
4,14591327,Columbia,14,2015-01-07,Utility,US20150220832A1,2015-08-06,9171249,2015-10-27,SYSTEMS AND METHODS FOR IDENTIFICATION OF SPIK...,SMALL,false,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022
3812,18090083,Columbia,18,2022-12-28,Utility,US20230148096A1,2023-05-11,,,YEAST-BASED VACCINES,SMALL,Other,2022
3813,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022
3814,18091615,Rowan,18,2022-12-30,Utility,US20230140691A1,2023-05-04,,,"OPTICALLY CLEAR, IN-SITU FORMING BIODEGRADABLE...",SMALL,true,2022


In [126]:
# delete application dataframe 
del appl_df 

In [127]:
# print first five rows of data (use transpose for readability)
nj_univ_df.head(5).transpose()

,0,1,2,3,4
application_number,14588788,14589306,14589845,62099988,14591327
univ,Columbia,Columbia,Princeton,Rutgers,Columbia
series,14,14,14,62,14
filing_date,2015-01-02,2015-01-05,2015-01-05,2015-01-05,2015-01-07
application_invention_type,Utility,Utility,Utility,Provisional,Utility
earliest_pgpub_number,US20150230724A1,US20160231649A1,US20150191765A1,,US20150220832A1
earliest_pgpub_date,2015-08-20,2016-08-11,2015-07-09,,2015-08-06
patent_number,,9454077,9752175,,9171249
patent_issue_date,,2016-09-27,2017-09-05,,2015-10-27
invention_title,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,Photochemical Methods and Photoactive Compound...,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,Sustainable Supply of Recipe Components for Ce...,SYSTEMS AND METHODS FOR IDENTIFICATION OF SPIK...


In [128]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,SMALL,false,2015
1,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,SMALL,false,2015
2,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015
3,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015
4,14591327,Columbia,14,2015-01-07,Utility,US20150220832A1,2015-08-06,9171249,2015-10-27,SYSTEMS AND METHODS FOR IDENTIFICATION OF SPIK...,SMALL,false,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022
3812,18090083,Columbia,18,2022-12-28,Utility,US20230148096A1,2023-05-11,,,YEAST-BASED VACCINES,SMALL,Other,2022
3813,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022
3814,18091615,Rowan,18,2022-12-30,Utility,US20230140691A1,2023-05-04,,,"OPTICALLY CLEAR, IN-SITU FORMING BIODEGRADABLE...",SMALL,true,2022


In [129]:
nj_univ_df.to_csv('valid.csv')

In [130]:
# set filepath to PatEx inventor data csv file
filepath = os_path.join(PATEX_DIR, 'all_inventors.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\all_inventors.csv


In [131]:
# open csv file as a pandas dataframe
# - identify the header as row 0 (the first row)
# - use all columns (do not specify usecols)
# - ensure all columns are read in as strings (important for the application number)
# - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
inv_df = pd.read_csv(filepath,
                     header=0,
                     dtype=str,
                     keep_default_na=False)

In [132]:
inv_df.columns

Index(['application_number', 'inventor_name_first', 'inventor_name_middle',
       'inventor_name_last', 'inventor_rank', 'inventor_city_name',
       'inventor_region_code', 'inventor_country_code'],
      dtype='object')

In [133]:
inv_df.head()

,application_number,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code
0,04840815,WILLIAM,D.,SCHAEFFER,1,POMONA,CA,US
1,05434252,HERMAN,,LEVIN,1,GLENVIEW,IL,US
2,05434252,ERIC,K.,MAXON,2,EVANSTON,IL,US
3,05603052,AGIS,F.,KYDONIEUS,1,NEW YORK,NY,US
4,05823902,WILLIAM,G.,HECKENHAUER,1,BUCYRUS,OH,US


In [134]:
# left join applicant_df (left table) to inv_df (right table); use indicator=True to check results 
nj_univ_appl_inv_df = nj_univ_df.merge(inv_df, 
                                       how='left', 
                                       on='application_number', 
                                       indicator=True)

In [135]:
nj_univ_appl_inv_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,...,aia_first_to_file,filing_year,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code,_merge
0,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,...,false,2015,Allen,E.,WAZIRI,1,Denver,CO,US,both
1,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,...,false,2015,Ronald,G.,EMERSON,2,Chappaqua,NY,US,both
2,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,...,false,2015,Guy,M.,MCKHANN,3,Bronxville,NY,US,both
3,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,...,false,2015,Jeffrey,T.,KOBERSTEIN,1,Storrs,CT,US,both
4,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,...,false,2015,Nicholas,J.,TURRO,2,Tenafly,NJ,US,both


In [136]:
nj_univ_appl_inv_df.shape

(12511, 21)

In [137]:
merge_results_series = nj_univ_appl_inv_df['_merge'].value_counts()
print(merge_results_series)

print('\nMerge results: ')
print('Number application-applicant observations with inventor data: {:,}'.
      format(merge_results_series['both']))
print('Number application-applicant observations missing inventor data: {:,}'.
      format(merge_results_series['left_only']))

both          12494
left_only        17
right_only        0
Name: _merge, dtype: int64

Merge results: 
Number application-applicant observations with inventor data: 12,494
Number application-applicant observations missing inventor data: 17


In [138]:
# keep _merge=='both'
nj_univ_appl_inv_df = nj_univ_appl_inv_df.loc[nj_univ_appl_inv_df['_merge']=='both']

In [139]:
# drop _merge column 
nj_univ_appl_inv_df.drop(columns=['_merge'], inplace=True)

In [140]:
nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].count()

univ
Columbia     5773
NJIT          594
Princeton    2157
Rowan         313
Rutgers      3083
Seton          23
Stevens       551
Name: application_number, dtype: int64

In [161]:
nj_univ_appl_inv_df['full_name'] = nj_univ_appl_inv_df['inventor_name_first'] + ' ' + nj_univ_appl_inv_df['inventor_name_middle'].fillna('') + ' ' + nj_univ_appl_inv_df['inventor_name_last']
nj_univ_appl_inv_df['full_name'] = nj_univ_appl_inv_df['full_name'].str.strip()

In [162]:
nj_univ_appl_inv_df['inv_id'] = nj_univ_appl_inv_df.groupby(['full_name']).ngroup()

In [163]:
type(nj_univ_appl_inv_df['series'][0])

str

In [164]:
filtered_nj_univ_appl_inv_df = nj_univ_appl_inv_df[nj_univ_appl_inv_df['series'].isin(['60', '61', '62', '63', '64'])]

In [165]:
nonprov_nj_univ_appl_inv_df = nj_univ_appl_inv_df[~nj_univ_appl_inv_df['series'].isin(['60', '61', '62', '63', '64'])]

In [166]:
#filtered_nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].value_count()
filtered_nj_univ_appl_inv_df[filtered_nj_univ_appl_inv_df['univ'] == 'Seton'].groupby(by=['univ'])['application_number'].value_counts()

univ   application_number
Seton  62637445              3
       62532118              2
Name: application_number, dtype: int64

In [167]:
filtered_nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].nunique()

univ
Columbia     625
NJIT          78
Princeton    128
Rowan         47
Rutgers      310
Seton          2
Stevens       66
Name: application_number, dtype: int64

In [168]:
# count number of unque inventors by university
nonprov_nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].nunique()

univ
Columbia     1130
NJIT          115
Princeton     487
Rowan          71
Rutgers       632
Seton           6
Stevens       102
Name: application_number, dtype: int64

In [169]:
nonprov_nj_univ_appl_inv_df.to_csv('final_results.csv')

In [170]:
# import gender_it_functions
from gender_it import gender_it_functions as gf

In [171]:
#gf.read_wgnd(path='gender_it//')

In [172]:
# get gender attributions for inventors; set threshold to 0.60
nj_univ_appl_inv_gender_df = gf.get_gender(nonprov_nj_univ_appl_inv_df, 
                                           name_column = 'inventor_name_first', 
                                           country_column = 'inventor_country_code', 
                                           treshold = 0.6)

Step 1 - reading the name-country-gender dictionary
reading the dictionnary.
Step 2 - reading the name-language-gender dictionary
reading the dictionnary.
downloading the dictionnary.
Step 3 - reading the name-gender dictionary.
reading the dictionnary.
Results distirbution is as follow: 
            gender  Percentage
M            5865   67.228336
F            1763   20.208620
not found    1082   12.402568
?              14    0.160477


In [173]:
nj_univ_appl_inv_gender_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,...,inventor_city_name,inventor_region_code,inventor_country_code,inv_id,full_name,level,gender,F,M,?
4719,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,...,Denver,CO,US,223,Allen E. WAZIRI,2.0,M,0.0,1.0,0.0
7897,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,...,Chappaqua,NY,US,4179,Ronald G. EMERSON,2.0,M,0.0,1.0,0.0
5901,14588788,Columbia,14,2015-01-02,Utility,US20150230724A1,2015-08-20,,,SYSTEMS AND METHODS FOR MEASURING BRAIN ACTIVITY,...,Bronxville,NY,US,1738,Guy M. MCKHANN,2.0,M,0.0,1.0,0.0
6218,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,...,Storrs,CT,US,2204,Jeffrey T. KOBERSTEIN,2.0,M,0.0,1.0,0.0
7317,14589306,Columbia,14,2015-01-05,Utility,US20160231649A1,2016-08-11,9454077,2016-09-27,Photochemical Methods and Photoactive Compound...,...,Tenafly,NJ,US,3607,Nicholas J. TURRO,2.0,M,0.0,1.0,0.0


In [174]:
nj_univ_appl_inv_gender_df.columns

Index(['application_number', 'univ', 'series', 'filing_date',
       'application_invention_type', 'earliest_pgpub_number',
       'earliest_pgpub_date', 'patent_number', 'patent_issue_date',
       'invention_title', 'small_entity_indicator', 'aia_first_to_file',
       'filing_year', 'inventor_name_first', 'inventor_name_middle',
       'inventor_name_last', 'inventor_rank', 'inventor_city_name',
       'inventor_region_code', 'inventor_country_code', 'inv_id', 'full_name',
       'level', 'gender', 'F', 'M', '?'],
      dtype='object')

In [175]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","gender","filing_year"]).size()

univ      gender     filing_year
Columbia  ?          2018            1
                     2020            3
                     2021            2
          F          2015           90
                     2016           77
                                    ..
Stevens   not found  2018            2
                     2019           10
                     2020            8
                     2021            3
                     2022           13
Length: 155, dtype: int64

In [200]:
combined_df = nj_univ_appl_inv_gender_df.groupby(by=["univ","application_number","invention_title",'full_name',"gender","filing_year"])['inv_id'].nunique()
#combined_df = nj_univ_appl_inv_gender_df.groupby(by=['full_name'])['inv_id'].nunique()

combined_df = combined_df.reset_index()

In [201]:
combined_df.to_csv('combined_results.csv')

In [202]:
#nj_univ_appl_inv_gender_df.groupby(by=["univ","application_invention_type","filing_year","gender"])['inv_id'].nunique()

In [203]:
#nj_univ_appl_inv_gender_df.groupby(by=["univ","application_invention_type","filing_year","gender"])['application_number'].nunique()

In [204]:
#nj_univ_appl_inv_gender_df.to_csv('first_results_with_gender.csv')

In [205]:
#print(appl_dict)

In [206]:
print(combined_df.columns)

Index(['univ', 'application_number', 'invention_title', 'full_name', 'gender',
       'filing_year', 'inv_id'],
      dtype='object')


In [207]:
NJIT_yearwise_df = combined_df[combined_df["univ"] == "NJIT"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()
Rutgers_yearwise_df = combined_df[combined_df["univ"] == "Rutgers"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()
Rowan_yearwise_df = combined_df[combined_df["univ"] == "Rowan"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()
Princeton_yearwise_df = combined_df[combined_df["univ"] == "Princeton"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()
Seton_yearwise_df = combined_df[combined_df["univ"] == "Seton"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()
Stevens_yearwise_df = combined_df[combined_df["univ"] == "Stevens"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()
Columbia_yearwise_df = combined_df[combined_df["univ"] == "Columbia"].groupby(["filing_year",'application_number',"invention_title","full_name","gender"]).size()

NJIT_yearwise_df.to_csv('NJIT_yearwise_df.csv')
Rutgers_yearwise_df.to_csv('Rutgers_yearwise_df.csv')
Rowan_yearwise_df.to_csv('Rowan_yearwise_df.csv')
Princeton_yearwise_df.to_csv('Princeton_yearwise_df.csv')
Seton_yearwise_df.to_csv('Seton_yearwise_df.csv')
Stevens_yearwise_df.to_csv('Stevens_yearwise_df.csv')
Columbia_yearwise_df.to_csv('Columbia_yearwise_df.csv')

In [209]:
#test1 = nj_univ_appl_inv_gender_df[nj_univ_appl_inv_gender_df["univ"] == "Seton"].groupby(["filing_year",'application_number',"invention_title",'inventor_name_first', 'inventor_name_middle',
      # 'inventor_name_last',"gender"]).size()
#test1